In [30]:
from langchain_community.document_loaders import WebBaseLoader, PyPDFDirectoryLoader, PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable

from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate

from langchain.chains import retrieval_qa

from langchain.llms import Ollama
from langchain_groq import ChatGroq

from dotenv import load_dotenv
import os

In [13]:
str = input("Enter link: ")
loader = WebBaseLoader(str)

In [22]:
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
final_docs = text_splitter.split_documents(documents)
title = final_docs[0].metadata['title']

In [20]:
load_dotenv()
groq_api_key = os.environ['GROQ_API_KEY']
llm = ChatGroq(api_key=groq_api_key, model="gemma2-9b-it", temperature=0.4)

In [32]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a neutralizer assistant that converts news headline into a neutral, bias-free search query suitable for finding balanced articles on the topic. Focus only on the core subject matter."),
    ("human", "Headline {headline}"),
    ("system", "Neutral Query: ")
])

In [33]:
chain = prompt | llm
response = chain.invoke({"headline" : title})

In [35]:
response.content

'Man missing after IndiGo flight incident \n'